In [1]:
import pandas as pd


In [5]:
#Source:https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download and importing IMDB dataset(make sure you keep it in your working directory
messages=pd.read_csv("IMDB Dataset.csv", names=["review","sentiment"])

In [6]:
messages

,review,sentiment
0,review,sentiment
1,One of the other reviewers has mentioned that ...,positive
2,A wonderful little production. <br /><br />The...,positive
3,I thought this was a wonderful way to spend ti...,positive
4,Basically there's a family where a little boy ...,negative
...,...,...
49996,I thought this movie did a down right good job...,positive
49997,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49998,I am a Catholic taught in parochial elementary...,negative
49999,I'm going to have to disagree with the previou...,negative


In [7]:
messages.shape

(50001, 2)

In [8]:
#Data Cleaning and Preprocessing
import re
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shrey_98/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [12]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z0-9]',' ', messages['review'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word)for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [17]:
#Now creating a bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2000,binary=True,ngram_range=(2,2))

In [24]:
X=cv.fit_transform(corpus).toarray()

In [25]:
X[1]

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
y=pd.get_dummies(messages['sentiment'],dtype=int)
y=y.iloc[:,1].values

In [27]:
y

array([0, 1, 1, ..., 0, 0, 0])

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.25)

In [30]:
X_train,y_train

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([0, 1, 1, ..., 1, 1, 0]))

In [33]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB().fit(X_train,y_train)

In [34]:
y_pred=model.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))


0.7862570994320455
              precision    recall  f1-score   support

           0       0.76      0.80      0.78      5934
           1       0.81      0.77      0.79      6567

    accuracy                           0.79     12501
   macro avg       0.79      0.79      0.79     12501
weighted avg       0.79      0.79      0.79     12501



In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
A=tf.fit_transform(corpus).toarray()

In [57]:
b=pd.get_dummies(messages['sentiment'],dtype=int)
b=b.iloc[:,1].values

In [59]:
from sklearn.model_selection import train_test_split
A_train,A_test,b_train,b_test=train_test_split(A,b,random_state=0,test_size=0.25)

In [60]:
A_train,b_train

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 1, 1, ..., 1, 1, 0]))

In [61]:
model1=MultinomialNB().fit(A_train,b_train)

In [62]:
b_pred=model1.predict(A_test)

In [63]:
print(accuracy_score(b_pred,b_test))
print(classification_report(b_pred,b_test))


0.8463322934165267
              precision    recall  f1-score   support

           0       0.83      0.86      0.84      6023
           1       0.86      0.84      0.85      6478

    accuracy                           0.85     12501
   macro avg       0.85      0.85      0.85     12501
weighted avg       0.85      0.85      0.85     12501



In [64]:
from sklearn.ensemble import RandomForestClassifier
cf=RandomForestClassifier()

In [65]:
cf.fit(A_train,b_train)

RandomForestClassifier()

In [66]:
b_pred=cf.predict(A_test)

In [67]:
print("Random Forest",accuracy_score(b_pred,b_test))
print(classification_report(b_pred,b_test))


Random Forest 0.8424126069914407
              precision    recall  f1-score   support

           0       0.85      0.84      0.84      6334
           1       0.83      0.85      0.84      6167

    accuracy                           0.84     12501
   macro avg       0.84      0.84      0.84     12501
weighted avg       0.84      0.84      0.84     12501



In [68]:
#Now doing it with your own Word-2Vec model
!pip install gensim

In [69]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess


In [70]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [77]:
import gensim

In [78]:
models=gensim.models.Word2Vec(words,window=5,min_count=2)

In [81]:
#Computed with cosine similarity, closer to 1 means most similar
models.wv.similar_by_word('action')

[('suspens', 0.5560059547424316),
 ('fight', 0.5543193817138672),
 ('dramat', 0.5140547752380371),
 ('pace', 0.48967012763023376),
 ('gori', 0.48706570267677307),
 ('heist', 0.4847753942012787),
 ('tens', 0.47410231828689575),
 ('thrill', 0.46656185388565063),
 ('gore', 0.4608564078807831),
 ('tension', 0.4562355875968933)]

In [82]:
import numpy as np

In [83]:
def avg_word2vec(doc):
    return np.mean([models.wv[word] for word in doc if word in models.wv.index_to_key],axis=0)

In [85]:
!pip install tqdm

In [86]:
from tqdm import tqdm

In [88]:
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))
    

100%|████████████████████████████████████| 50001/50001 [01:46<00:00, 468.19it/s]


In [89]:
type(X)

list

In [90]:
X_new=np.array(X)

In [91]:
X_new[5]

array([ 0.697471  ,  0.25811028, -0.23136681,  0.02151822, -0.1985191 ,
       -0.0675873 ,  0.2970444 , -0.04757069, -0.41342312,  0.19906022,
        0.68235236,  0.6132807 , -0.16784549,  0.44443858, -0.17382938,
        0.31310156,  0.29759577,  0.2051022 , -0.45551652, -0.81347007,
        0.07428182,  0.23729672,  0.43566763,  0.2401548 ,  0.18756443,
       -0.02043974, -0.47121465, -0.31881782,  0.19318058,  0.02939977,
       -0.30255312,  0.23965389,  0.40264133, -0.27470312, -0.11212132,
        0.03157189,  0.25323877, -0.45072004, -0.28427318, -0.17987029,
        0.07161406,  0.05314956,  0.1201514 , -0.13248716,  0.06088365,
        0.38455245, -0.45822343, -0.07228225,  0.03497584,  0.05671017,
       -0.78109974,  0.5457158 ,  0.07278495, -0.6740215 ,  0.62952363,
       -0.47215036, -0.08786225,  0.03973249, -0.124457  ,  0.06486946,
        0.46781605, -0.6158219 , -0.11731587, -0.4557278 , -0.24794905,
        0.03788794,  0.20369343,  0.58233714,  0.20491543,  0.23

In [93]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.25,random_state=0)

In [94]:
X_train,y_train

(array([[ 0.68491167,  0.30583858, -0.53512377, ...,  0.00288718,
          0.11056052,  0.46941945],
        [ 0.5120286 ,  0.4306038 , -0.23731719, ..., -0.04037752,
         -0.02617848,  0.41945928],
        [ 0.39555386,  0.1259601 , -0.31451383, ...,  0.27356488,
         -0.00787782,  0.5645078 ],
        ...,
        [ 0.90204525,  0.55935967, -0.36278   , ...,  0.19348133,
          0.18692282,  0.8573418 ],
        [ 0.4069974 ,  0.3723077 ,  0.10636804, ...,  0.16671117,
          0.08133376,  0.3913527 ],
        [ 0.23447193,  0.24421975, -0.20157489, ..., -0.40994626,
         -0.21702594,  0.46301463]], dtype=float32),
 array([0, 1, 1, ..., 1, 1, 0]))

In [96]:
model3=cf.fit(X_train,y_train)

In [ ]:
y_pred=model3.predict(X_test)

In [98]:
print("Accuracy with Random Forest for  Avg word2vec FeatureRepresentation for IMDB Dataset",accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

Accuracy with Random Forest for  Avg word2vec FeatureRepresentation for IMDB Dataset 0.7862570994320455
              precision    recall  f1-score   support

           0       0.76      0.80      0.78      5934
           1       0.81      0.77      0.79      6567

    accuracy                           0.79     12501
   macro avg       0.79      0.79      0.79     12501
weighted avg       0.79      0.79      0.79     12501

